- Read all available news from publico.pt.
- To choose only tech news change urlPublico to the following

```python
def urlPublico(year, month, day):
    return "https://arquivo.pt/wayback/cdx?url=publico.pt/{year:04d}/{month:02d}/{day:02d}/tecnologia&matchType=prefix&output=json".format(year = year, month = month, day = day)
```

- To get news from different dates change start_date and/or end_date

In [16]:
import requests, datetime, time, json, joblib, re

def urlPublico(year, month, day):
    return "https://arquivo.pt/wayback/cdx?url=publico.pt/{year:04d}/{month:02d}/{day:02d}/&matchType=prefix&output=json".format(year = year, month = month, day = day)

def getOutput(date):
    r = requests.get(url = urlPublico(date.year, date.month, date.day))
    if (r.status_code == 200 and r.text != ""):
        for line in r.text.split('\n'):
            if (line != ""):
                output.append(json.loads(line))
    elif (r.status_code == 429):
        print("Reached threshold. Waiting 10 seconds")
        time.sleep(10)
        getOutput(date)

websites = [
    "publico.pt"
]

output = []

start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2021, 11, 1)
dates = []

while (start_date < end_date):
    dates.append(start_date)
    start_date += datetime.timedelta(days = 1)

#joblib.Parallel(n_jobs=5)(joblib.delayed(getOutput)(date) for date in dates)
for date in dates:
    if (date.day == 1):
        print(date.month)
    getOutput(date)
    
results = {"publico" : output}

1
2
3
4
5
6
7
Reached threshold. Waiting 10 seconds
Reached threshold. Waiting 10 seconds
8
9
10


In [28]:
def noticiasAoMinuto():
    return requests.get(url = "https://arquivo.pt/wayback/cdx?url=www.noticiasaominuto.com/&matchType=prefix&output=json&from=20210101000000&to=20211101000000")

output = []
r = noticiasAoMinuto()

if (r.status_code == 429):
    time.sleep(10)
    r = noticiasAoMinuto()

if (r.status_code == 200):
    for line in r.text.split('\n'):
        if (line.startswith('{') and line.endswith('}')):
            output.append(json.loads(line))

results['noticiasAoMinuto'] = output

In [31]:
# print number of news on a site
for identifier, result in results.items():
    print("site: {0}\t noticias: {1}".format(identifier, len(result)))

print(results['noticiasAoMinuto'])

site: publico	 noticias: 23382
site: noticiasAoMinuto	 noticias: 4613
[{'urlkey': 'com,noticiasaominuto)/', 'timestamp': '20210108103250', 'url': 'https://www.noticiasaominuto.com/', 'mime': 'text/html', 'status': '200', 'digest': 'NQDNXTOFA7VXSF5QU3VLUNASLXVPXSHE', 'length': '27509', 'offset': '66726128', 'filename': 'WEB-20210108103228326-p100.arquivo.pt.warc.gz', 'collection': 'EAWP36', 'source': '$root:EAWP36.cdxj', 'source-coll': '$root'}, {'urlkey': 'com,noticiasaominuto)/', 'timestamp': '20210111101701', 'url': 'https://www.noticiasaominuto.com/', 'mime': 'text/html', 'status': '200', 'digest': 'RBRTUYKJB6HSOQRJNS4JC2BCRXU7IVOB', 'length': '27227', 'offset': '48527831', 'filename': 'WEB-20210111101629087-p100.arquivo.pt.warc.gz', 'collection': 'EAWP36', 'source': '$root:EAWP36.cdxj', 'source-coll': '$root'}, {'urlkey': 'com,noticiasaominuto)/', 'timestamp': '20210111173829', 'url': 'https://www.noticiasaominuto.com/', 'mime': 'text/html', 'status': '200', 'digest': 'AVNB3MC33STJ

- Clear the news that were fetched.
- Multiple news with different timestamps are cleared.
- Some websites that are not news are cleared.
- Count the number of indexations for each of the news.

In [46]:
# Clear repeated news from publico.pt
news = {}
number_of_indexations = {}

for link in results["publico"]:
    splitted_url = link['url'].split('/')
    if (len(splitted_url) < 9): # not news
        continue
    if ("." in splitted_url[8]): # not news
        continue
    key = link['urlkey']
    if (news.get(key) == None):
        news[key] = link
        number_of_indexations[key] = 1
    else:
        number_of_indexations[key] += 1

pattern = re.compile("[0-9]+")

for link in results["noticiasAoMinuto"]:
    splitted_url = link['url'].split('/')
    if (len(splitted_url) < 6): # not news
        continue
    if (not pattern.match(splitted_url[4])):
        continue
    key = link['urlkey']
    if (news.get(key) == None):
        news[key] = link
        number_of_indexations[key] = 1
    else:
        number_of_indexations[key] += 1


In [50]:
print(len(news))

9282
{'urlkey': 'pt,publico)/2021/01/01/culturaipsilon/noticia/amigo-cunhal-compagnon-route-pcp-apoiante-costa-1944794', 'timestamp': '20210101201956', 'url': 'https://www.publico.pt/2021/01/01/culturaipsilon/noticia/amigo-cunhal-compagnon-route-pcp-apoiante-costa-1944794', 'mime': 'text/html', 'status': '200', 'digest': 'J4UYV2C25UFRT3QUBCSLGWIDV6J6WMLO', 'length': '75414', 'offset': '39345905', 'filename': 'raq-projetos-fct-2019-20210101201455914-01267-12kmyp7e.warc.gz', 'collection': 'EAWP34', 'source': '$root:EAWP34.cdxj', 'source-coll': '$root'}
35


- Fetch the website for the links obtained.
- Some sites need to be cleaned (they are not news).

In [37]:
websites = {}

def create_website_link(url, timestamp):
    return "https://arquivo.pt/noFrame/replay/{0}/{1}".format(timestamp, url)

for key, link in news.items():
    response = requests.get(create_website_link(link['url'], link['timestamp']))
    websites[key] = response.text

# len between 372500 and 37599 might be 404 page (publico.pt)


KeyboardInterrupt: 

- In the end we have 3 dictionaries with the same keys (key of the news).
- News has all the info provided in the initial API requests.
- Number_of_indexations has the number of different timestamps for each website.
- Websites contains the HTML for each of the websites.

In [ ]:
print(len(websites))
print(news['amigo-cunhal-compagnon-route-pcp-apoiante-costa-1944794'])
print(number_of_indexations['amigo-cunhal-compagnon-route-pcp-apoiante-costa-1944794'])
print(websites['amigo-cunhal-compagnon-route-pcp-apoiante-costa-1944794'])